## Preprocessing

In [36]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [37]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])


In [38]:
# Step 2: Determine the target(s) and features
target = application_df["IS_SUCCESSFUL"]
features = application_df.drop(columns=["IS_SUCCESSFUL"])
features

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000


In [39]:
# Determine the number of unique values in each column.
unique_counts = features.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
dtype: int64

In [40]:
# Identify columns with more than 10 unique values
high_cols = [col for col in features.columns if unique_counts[col] > 10]
high_cols


['APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT']

In [41]:
for column in features.columns:
    if unique_counts[column] > 10:
        print(f"Unique values for {column}:")
        print(features[column].value_counts())
        print()

Unique values for APPLICATION_TYPE:
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

Unique values for CLASSIFICATION:
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

Unique values for ASK_AMT:
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64



In [46]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
cutoff = 1000
for column in features.columns:
    if unique_counts[column] > 10:
        value_counts = features[column].value_counts()
        rare_values = value_counts[value_counts < cutoff].index
        features[column] = features[column].apply(lambda x: "Other" if x in rare_values else x)



In [45]:
# Choose a cutoff value and create a list of application types to be replaced
# I have replaced both the application type and classification in one code 
for column in features.columns:
    if unique_counts[column] > 10:
        print(f"Unique values for {column} after replacing rare values:")
        print(features[column].value_counts())
        print()

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

Unique values for APPLICATION_TYPE after replacing rare values:
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

Unique values for CLASSIFICATION after replacing rare values:
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

Unique values for ASK_AMT after replacing rare values:
5000     25398
Other     8901
Name: ASK_AMT, dtype: int64



T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [49]:
features.head(9)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,Other
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,Other
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,Other
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,Other
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,Other
8,Other,Independent,C1000,ProductDev,Trust,1,1-9999,N,Other


In [51]:
# Convert categorical data to numeric with `pd.get_dummies`
features_encoded = pd.get_dummies(features)
features_encoded

,STATUS,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Other
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
4,1,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
34295,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
34296,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
34297,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0


In [52]:
# Split our preprocessed data into our features and target arrays
X = features_encoded
y = target

# Split the preprocessed data into a training and testing dataset
# Use the train_test_split function to split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [55]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_features = len(X_train_scaled[0])


model = tf.keras.models.Sequential()

# First hidden layer
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 30
model.add(Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
model.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

/Users/Masih/anaconda3/envs/devv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 50)             │         2,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,661 (14.30 KB)

 Trainable params: 3,661 (14.30 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
# Define a custom callback to save the model weights every five epochs
class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 5 == 0:  # Save weights every five epochs
            self.model.save_weights("AlphabetSoupCharity_epoch_{}.weights.h5".format(epoch + 1))

# Create an instance of the custom callback
cp_callback = CustomModelCheckpoint()


In [69]:
#train the model 
epochs = 100  
model.fit(X_train_scaled, y_train, epochs=epochs, callbacks=[cp_callback])

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7365 - loss: 0.5434
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7382 - loss: 0.5441
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7374 - loss: 0.5446
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7329 - loss: 0.5505
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7437 - loss: 0.5365
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7335 - loss: 0.5463
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7331 - loss: 0.5467
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7350 - loss: 0.5433
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7376 - loss: 0.5446
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7433 - loss: 0.5389
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7330 - loss: 0.5463
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [70]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 2ms/step - accuracy: 0.7287 - loss: 0.5637
Loss: 0.5636867880821228, Accuracy: 0.7287463545799255


In [71]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity.h5")